# 01-10: TypeScript 装饰器 (Decorators)装饰器是一种特殊的声明，可以被附加到类声明、方法、访问器、属性或参数上。

In [ ]:
// 启用装饰器需要在 tsconfig.json 中设置:
// "experimentalDecorators": true
// "emitDecoratorMetadata": true
// ========== 1. 类装饰器 ==========
function Logger(constructor: Function) {
  console.log('Logging...');
  console.log(constructor);
}
@Logger
class Person {
  name = 'Max';
  
  constructor() {
    console.log('Creating person object...');
  }
}
const person = new Person();

In [ ]:
// ========== 2. 装饰器工厂 ==========
// 可以返回一个函数的函数，允许传参
function Logger(logString: string) {
  return function(constructor: Function) {
    console.log(logString);
    console.log(constructor);
  };
}
@Logger('LOGGING - PERSON')
class Person2 {
  name = 'Max';
  
  constructor() {
    console.log('Creating person object...');
  }
}

In [ ]:
// ========== 3. 方法装饰器 ==========
function MethodDecorator(
  target: any, 
  propertyKey: string, 
  descriptor: PropertyDescriptor
) {
  console.log('Method decorator!');
  console.log('Target:', target);
  console.log('Property Key:', propertyKey);
  console.log('Descriptor:', descriptor);
}
class Product {
  title: string;
  private _price: number;
  
  constructor(t: string, p: number) {
    this.title = t;
    this._price = p;
  }
  
  @MethodDecorator
  getPriceWithTax(tax: number) {
    return this._price * (1 + tax);
  }
}

In [ ]:
// ========== 4. 属性装饰器 ==========
function PropertyDecorator(target: any, propertyKey: string) {
  console.log('Property decorator!');
  console.log('Target:', target);
  console.log('Property Key:', propertyKey);
}
class Product2 {
  @PropertyDecorator
  title: string;
  @PropertyDecorator
  _price: number;
  
  constructor(t: string, p: number) {
    this.title = t;
    this._price = p;
  }
}

In [ ]:
// ========== 5. 参数装饰器 ==========
function ParameterDecorator(
  target: any, 
  methodName: string, 
  parameterIndex: number
) {
  console.log('Parameter decorator!');
  console.log('Target:', target);
  console.log('Method Name:', methodName);
  console.log('Parameter Index:', parameterIndex);
}
class Product3 {
  getPriceWithTax(@ParameterDecorator tax: number) {
    return 100 * (1 + tax);
  }
}

In [ ]:
// ========== 6. 实际应用: 自动绑定 this ==========
function Autobind(
  _target: any, 
  _methodName: string, 
  descriptor: PropertyDescriptor
) {
  const originalMethod = descriptor.value;
  const adjDescriptor: PropertyDescriptor = {
    configurable: true,
    enumerable: false,
    get() {
      const boundFn = originalMethod.bind(this);
      return boundFn;
    }
  };
  return adjDescriptor;
}
class Printer {
  message = 'This works!';
  
  @Autobind
  showMessage() {
    console.log(this.message);
  }
}
const p = new Printer();
const button = { addEventListener: (_: string, fn: Function) => fn() };
button.addEventListener('click', p.showMessage); // 输出: This works!

In [ ]:
// ========== 7. 实际应用: 验证装饰器 ==========
interface ValidatorConfig {
  [property: string]: string[]; // ['required', 'positive']
}
const registeredValidators: ValidatorConfig = {};
function Required(target: any, propName: string) {
  registeredValidators[target.constructor.name] = {
    ...registeredValidators[target.constructor.name],
    [propName]: [...(registeredValidators[target.constructor.name]?.[propName] || []), 'required']
  };
}
function PositiveNumber(target: any, propName: string) {
  registeredValidators[target.constructor.name] = {
    ...registeredValidators[target.constructor.name],
    [propName]: [...(registeredValidators[target.constructor.name]?.[propName] || []), 'positive']
  };
}
function validate(obj: any): boolean {
  const objValidatorConfig = registeredValidators[obj.constructor.name];
  if (!objValidatorConfig) return true;
  
  let isValid = true;
  for (const prop in objValidatorConfig) {
    for (const validator of objValidatorConfig[prop]) {
      switch (validator) {
        case 'required':
          isValid = isValid && !!obj[prop];
          break;
        case 'positive':
          isValid = isValid && obj[prop] > 0;
          break;
      }
    }
  }
  return isValid;
}
class Course {
  @Required
  title: string;
  @PositiveNumber
  price: number;
  
  constructor(t: string, p: number) {
    this.title = t;
    this.price = p;
  }
}
const courseForm = new Course('', -1);
console.log('Is valid:', validate(courseForm)); // false
const courseForm2 = new Course('TypeScript', 100);
console.log('Is valid:', validate(courseForm2)); // true